Confirmed by Omar

the project is about (Synthetic Financial Datasets For Fraud Detection
)  .. https://www.kaggle.com/datasets/ealaxi/paysim1

In [6]:
# Check GPU availability
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.metrics import accuracy_score, confusion_matrix

# Import GPU-based (cuML) libraries
import cudf
import cuml
from cuml.linear_model import LogisticRegression
from cuml.model_selection import train_test_split
from cuml.preprocessing import StandardScaler


/usr/local/lib/python3.11/dist-packages/cudf/utils/_ptxcompiler.py:64: UserWarning: Error getting driver and runtime versions:

stdout:



stderr:

Traceback (most recent call last):
  File "<string>", line 4, in <module>
  File "/usr/local/lib/python3.11/dist-packages/numba_cuda/numba/cuda/cudadrv/driver.py", line 314, in __getattr__
    raise CudaSupportError("Error at driver init: \n%s:" %
numba.cuda.cudadrv.error.CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
:


Not patching Numba
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.11/dist-packages/cudf/utils/gpu_utils.py:62: UserWarning: Failed to dlopen libcuda.so.1
  warnings.warn(str(e))


In [3]:
pip install kagglehub


In [5]:
import os
import cudf
import kagglehub

# Download dataset
path = kagglehub.dataset_download("ealaxi/paysim1")

import os
print(os.listdir(path))
# List files in the downloaded folder
print("Files in dataset directory:", os.listdir(path))

# Assuming the file is named something like:
# PS_20174392719_1491204439457_log.csv
csv_path = os.path.join(path, "PS_20174392719_1491204439457_log.csv")

# Load the dataset into a cuDF DataFrame
gdf = cudf.read_csv(csv_path)

# Display a sample
print(gdf.head())


100%|██████████| 178M/178M [00:01<00:00, 129MB/s]

Extracting files...


['PS_20174392719_1491204439457_log.csv']
Files in dataset directory: ['PS_20174392719_1491204439457_log.csv']


MemoryError: std::bad_alloc: CUDA error at: /pyenv/versions/3.12.9/lib/python3.12/site-packages/librmm/include/rmm/mr/device/cuda_memory_resource.hpp:62: cudaErrorInsufficientDriver CUDA driver version is insufficient for CUDA runtime version

In [ ]:
# Basic data exploration
print("\nDataset shape:", gdf.shape)
print("\nSample of the data:")
print(gdf.head())
print("\nSummary statistics:")
print(gdf.describe())
print("\nData types:")
print(gdf.dtypes)
print("\nClass distribution:")
print(gdf['isFraud'].value_counts())

In [ ]:
gdf = gdf.drop(['nameOrig', 'nameDest'], axis=1)
gdf['type'] = gdf['type'].astype('category').cat.codes
gdf['label'] = gdf['isFraud']
gdf = gdf.drop(['isFraud', 'isFlaggedFraud'], axis=1)

In [ ]:
X = gdf.drop('label', axis=1)
y = gdf['label']

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42, stratify=y)


In [ ]:
from cuml.linear_model import LogisticRegression
from cuml.metrics import accuracy_score

lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)


y_pred = lr_model.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print(f"Logistic Regression Accuracy: {acc:.4f}")

In [ ]:
from cuml.metrics import confusion_matrix
from sklearn.metrics import classification_report

cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)


print(classification_report(y_test.to_numpy(), y_pred.to_numpy()))

In [ ]:
from cuml.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100)
rf_model.fit(X_train, y_train)
rf_preds = rf_model.predict(X_test)
rf_acc = accuracy_score(y_test, rf_preds)
print(f"Random Forest Accuracy: {rf_acc:.4f}")


In [ ]:
from cuml.metrics import confusion_matrix
from sklearn.metrics import classification_report


cm = confusion_matrix(y_test, rf_preds)
print("Confusion Matrix:\n", cm)


print(classification_report(y_test.to_numpy(), rf_preds.to_numpy()))


In [ ]:
from cuml.neighbors import KNeighborsClassifier

# Train KNN model
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)


knn_preds = knn_model.predict(X_test)

knn_acc = accuracy_score(y_test, knn_preds)
print(f"KNN Accuracy: {knn_acc:.4f}")


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Logistic Regression
print(" Logistic Regression Report:")
print(classification_report(y_test.to_numpy(), y_pred.to_numpy(), digits=4))
print("Confusion Matrix:\n", confusion_matrix(y_test.to_numpy(), y_pred.to_numpy()))

# Random Forest
print("\n Random Forest Report:")
print(classification_report(y_test.to_numpy(), rf_preds.to_numpy(), digits=4))
print("Confusion Matrix:\n", confusion_matrix(y_test.to_numpy(), rf_preds.to_numpy()))

# KNN
print("\n KNN Report:")
print(classification_report(y_test.to_numpy(), knn_preds.to_numpy(), digits=4))
print("Confusion Matrix:\n", confusion_matrix(y_test.to_numpy(), knn_preds.to_numpy()))

In [ ]:
import time

# Logistic Regression
start = time.time()
lr_model.fit(X_train, y_train)
train_time_lr = time.time() - start

start = time.time()
lr_model.predict(X_test)
inference_time_lr = time.time() - start

# Random Forest
start = time.time()
rf_model.fit(X_train, y_train)
train_time_rf = time.time() - start

start = time.time()
rf_model.predict(X_test)
inference_time_rf = time.time() - start

# KNN
start = time.time()
knn_model.fit(X_train, y_train)
train_time_knn = time.time() - start

start = time.time()
knn_model.predict(X_test)
inference_time_knn = time.time() - start

print(f"""
 Training Times:
Logistic Regression: {train_time_lr:.4f} s
Random Forest: {train_time_rf:.4f} s
KNN: {train_time_knn:.4f} s

 Inference Times:
Logistic Regression: {inference_time_lr:.4f} s
Random Forest: {inference_time_rf:.4f} s
KNN: {inference_time_knn:.4f} s
""")


In [ ]:
import matplotlib.pyplot as plt


models = ['Logistic Regression', 'Random Forest', 'KNN']
accuracies = [acc, rf_acc, knn_acc]

plt.figure(figsize=(8, 5))
plt.bar(models, accuracies, color='steelblue')
plt.title("Model Accuracy Comparison")
plt.ylabel("Accuracy")
plt.ylim(0.90, 1.00)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()
